In [27]:
import pandas as pd
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, cohen_kappa_score
from sklearn.model_selection import KFold

In [28]:
Train = pd.read_csv('dataTrain.csv')
Test = pd.read_csv('dataTest.csv')

X_train = Train.drop('output', axis=1)
y_train = Train['output']

X_test = Test.drop('output', axis=1)
y_test = Test['output']

maxs = X_train.max().tolist()
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((241, 12), (61, 12), (241,), (61,))

In [29]:
def trainMLP(HLS):
    regr = MLPClassifier(hidden_layer_sizes=HLS,random_state=1, max_iter=5000)
    regr.fit(X_train, y_train)

    y_pred = regr.predict(X_test)

    print('')
    print('Architecture: {list}'.format(list = HLS))
    acc_score = accuracy_score(y_test, y_pred)
    print("Accuracy: {:.3f}".format(acc_score))
    rec_score = recall_score(y_test, y_pred)
    print("Recall: {:.3f}".format(rec_score))
    prec_score = precision_score(y_test, y_pred)
    print("Precision Score: {:.3f}".format(prec_score))
    F1_score = f1_score(y_test, y_pred)
    print("F1-Score: {:.3f}".format(F1_score))
    kappa = cohen_kappa_score(y_test, y_pred)
    print("Kappa Score: {:.3f}".format(kappa))

Architectures = [[5],[10], [5, 5], [5, 10],[10, 10]]

for i in Architectures: trainMLP(i)


Architecture: [5]
Accuracy: 0.869
Recall: 0.906
Precision Score: 0.853
F1-Score: 0.879
Kappa Score: 0.736

Architecture: [10]
Accuracy: 0.885
Recall: 0.875
Precision Score: 0.903
F1-Score: 0.889
Kappa Score: 0.770

Architecture: [5, 5]
Accuracy: 0.852
Recall: 0.812
Precision Score: 0.897
F1-Score: 0.852
Kappa Score: 0.706

Architecture: [5, 10]
Accuracy: 0.852
Recall: 0.844
Precision Score: 0.871
F1-Score: 0.857
Kappa Score: 0.705

Architecture: [10, 10]
Accuracy: 0.852
Recall: 0.781
Precision Score: 0.926
F1-Score: 0.847
Kappa Score: 0.707


In [30]:
def trainMLP(hidden_layer_sizes, X, y, n_splits=5):
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=1)
    
    accuracies = []
    recalls = []
    precisions = []
    f1_scores = []
    kappas = []
    
    for train_index, test_index in kf.split(X):
        X_train2, X_test2 = X.iloc[train_index], X.iloc[test_index]
        y_train2, y_test2 = y.iloc[train_index], y.iloc[test_index]
        
        regr = MLPClassifier(hidden_layer_sizes=hidden_layer_sizes, random_state=1, max_iter=5000)
        regr.fit(X_train2, y_train2)

        y_pred = regr.predict(X_test2)

        acc_score = accuracy_score(y_test2, y_pred)  
        rec_score = recall_score(y_test2, y_pred, average='weighted')  
        prec_score = precision_score(y_test2, y_pred, average='weighted')  
        F1_score = f1_score(y_test2, y_pred, average='weighted')
        kappa = cohen_kappa_score(y_test2, y_pred)

        accuracies.append(acc_score)
        recalls.append(rec_score)
        precisions.append(prec_score)
        f1_scores.append(F1_score)
        kappas.append(kappa)

    print(f'\nArchitecture: {hidden_layer_sizes}')
    print(f"Average Accuracy: {np.mean(accuracies):.3f} ± {np.std(accuracies):.3f}")
    print(f"Average Recall: {np.mean(recalls):.3f} ± {np.std(recalls):.3f}")
    print(f"Average Precision: {np.mean(precisions):.3f} ± {np.std(precisions):.3f}")
    print(f"Average F1-Score: {np.mean(f1_scores):.3f} ± {np.std(f1_scores):.3f}")
    print(f"Average Kappa: {np.mean(kappas):.3f} ± {np.std(kappas):.3f}")

Architectures = [[5], [10], [5, 5], [5, 10], [10, 10]]

for arch in Architectures:
    trainMLP(arch, X_train, y_train, n_splits=5)


Architecture: [5]
Average Accuracy: 0.821 ± 0.029
Average Recall: 0.821 ± 0.029
Average Precision: 0.837 ± 0.022
Average F1-Score: 0.818 ± 0.032
Average Kappa: 0.630 ± 0.059

Architecture: [10]
Average Accuracy: 0.776 ± 0.035
Average Recall: 0.776 ± 0.035
Average Precision: 0.781 ± 0.039
Average F1-Score: 0.774 ± 0.038
Average Kappa: 0.537 ± 0.079

Architecture: [5, 5]
Average Accuracy: 0.764 ± 0.053
Average Recall: 0.764 ± 0.053
Average Precision: 0.769 ± 0.051
Average F1-Score: 0.763 ± 0.052
Average Kappa: 0.515 ± 0.103

Architecture: [5, 10]
Average Accuracy: 0.772 ± 0.033
Average Recall: 0.772 ± 0.033
Average Precision: 0.776 ± 0.031
Average F1-Score: 0.771 ± 0.032
Average Kappa: 0.529 ± 0.066

Architecture: [10, 10]
Average Accuracy: 0.760 ± 0.087
Average Recall: 0.760 ± 0.087
Average Precision: 0.767 ± 0.090
Average F1-Score: 0.758 ± 0.087
Average Kappa: 0.505 ± 0.182
